In [9]:
import lxml.html as lx
import pandas as pd
import numpy as np
import requests
import requests_cache
import numpy as np
import nltk
import nltk.corpus
from nltk.corpus import wordnet
from itertools import chain
from plotnine import *
import time
import matplotlib.pyplot as plt
import argparse
%matplotlib inline

In [24]:
def get_page_url(base_url, page_url):
    requests_cache.install_cache('page_logs')
    cur_page = 1;
    urls = []
    while True:
        try:
            time.sleep(.05)
            response = requests.get(page_url+str(cur_page))
            html = lx.fromstring(response.text)
            html.make_links_absolute(base_url)
            url = html.xpath("//tbody//td/a/@href")
            urls += url
            cur_page +=1;
        except:
            break
    return urls

In [25]:
def unique(links):
    url = []
    for items in links:
        if items not in url:
            url.append(items)
    return url

In [26]:
def rank_lvl(op_url, name):
    requests_cache.install_cache("opna")
    time.sleep(.005)
    response = requests.get(op_url+name)
    response.raise_for_status()
    html = lx.fromstring(response.text)
    
    lvl = html.xpath("//*[contains(@class,'Level tip')]")
    recent_rank = html.xpath("//*[contains(@class, 'PastRankList')]/li[last()]")
    if len(lvl) > 0:
        lvl = lvl[0].text_content().strip()
    
    if len(recent_rank) > 0:
        recent_rank = recent_rank[0].text_content().strip()
        
    if len(lvl) > 0 and len(recent_rank) >0:
        return recent_rank,lvl
    
    elif len(lvl)>0:
        return np.nan,lvl
    else:
        return np.nan, np.nan 

In [38]:
def page_info(url,op_url):
    dates = []
    names = []
    members = []
    texts = []
    lvl = []
    rank = []
    requests_cache.install_cache("page_posts")
    try:
        response = requests.get(url)
        response.raise_for_status()
        html = lx.fromstring(response.text)
        time.sleep(.1)
        title = html.xpath("//*[contains(@class, 'float-left')]/h3")[0].text_content()
        all_dates = html.xpath("//*[contains(@class, 'post-header padding-side-10 clearfix')]")
        all_users = html.xpath("//*[contains(@class, 'post-user')]")
        member_type = html.xpath("//*[contains(@class, 'post-col-left')]//p")[1::2]
        for i in range(len(all_dates)):
            dates.append(all_dates[i].text_content().strip())
            names.append(all_users[i].text_content().strip())
            members.append(member_type[i].text_content().strip())
            texts.append(html.xpath("//*[contains(@class,'post-message')]//p")[i].text_content().strip())
        for name in names:
            op_na = rank_lvl(op_url,name)
            rank.append(op_na[0])
            lvl.append(op_na[1])
        return pd.DataFrame({'user_name':names,"tier": rank, "level": lvl, "tier": rank, "level": lvl, 'member_type': members, 'dates': dates,'title': title, 'text':texts})
    except:
        return

In [39]:
#NA 
na_base_url = "http://forums.na.leagueoflegends.com/"
na_extension = "board/forumdisplay.php?f=59&page="
na_op_url = "https://na.op.gg/summoner/userName="

#EUW
euw_base_url = "http://forums.euw.leagueoflegends.com/"
euw_extension = "board/forumdisplay.php?f=85&page="
euw_op_url = "https://euw.op.gg/summoner/userName="

#EUNE
eune_base_url = "http://forums.eune.leagueoflegends.com/"
eune_extension = "board/forumdisplay.php?f=85&page="
eune_op_url = "https://eune.op.gg/summoner/userName="

#OCE
oce_base_url = "http://forums.oce.leagueoflegends.com/"
oce_extension = "board/forumdisplay.php?f=59&page="
oce_op_url = "https://oce.op.gg/summoner/userName="


In [40]:
def region_pages_info(region_base_url, region_extension, region_op_url):
    region_pages_url = get_page_url(region_base_url, region_base_url + region_extension)
    region_pages_url = unique(region_pages_url)
    print(region_pages_url[:5])
    
    return [page_info(url, region_op_url) for url in region_pages_url[:3]]

In [41]:
euw = region_pages_info(euw_base_url,euw_extension,euw_op_url)
# eune = region_pages_info(eune_base_url,eune_extension,eune_op_url)
# na = region_pages_info(na_base_url, na_extension, na_op_url)
# oce = region_pages_info(oce_base_url,oce_extension,oce_op_url)
euw

['http://forums.euw.leagueoflegends.com/board/showthread.php?t=788232', 'http://forums.euw.leagueoflegends.com/board/showthread.php?t=536799', 'http://forums.euw.leagueoflegends.com/board/showthread.php?t=1622322', 'http://forums.euw.leagueoflegends.com/board/showthread.php?t=2104510', 'http://forums.euw.leagueoflegends.com/board/showthread.php?t=2104933']


[         user_name       tier level                   member_type       dates  \
 0        Riot Lyte        NaN    30  Lead Social Systems Designer  07-28-2012   
 1        Riot Lyte        NaN    30  Lead Social Systems Designer  07-28-2012   
 2        Riot Lyte        NaN    30  Lead Social Systems Designer  07-28-2012   
 3   RiotKiddington        NaN   NaN             Platform Engineer  07-29-2012   
 4          smitske        NaN    30                        Member  07-29-2012   
 5  SirKibaTheIIIrd        NaN    16                 Senior Member  07-29-2012   
 6  SinisterMrCream        NaN    30                 Senior Member  07-30-2012   
 7         baccaraa    S4 Gold    89                 Senior Member  07-31-2012   
 8          Konerai  S7 Silver    30                        Member  08-01-2012   
 9        Riot Lyte        NaN    30  Lead Social Systems Designer  08-07-2012   
 
                           title  \
 0  Tribunal FAQ [Updated 18/06]   
 1  Tribunal FAQ [Update

In [53]:
#Turn into dataframe
euw_df = pd.concat(euw)
eune_df = pd.concat(eune)
na_df = pd.concat(na)
oce_df = pd.concat(oce)

In [58]:
#Merge data
all_regions = pd.concat([na_df,euw_df,eune_df,oce_df]).reset_index()
all_regions.columns.values[0] = 'tree'
all_regions.to_dense().to_csv("pages_forum.csv", index = False, sep=',', encoding='utf-8')